Example notebook

To do: DATA CONFIG BUILDER

Walk through a folder, collecting all the files into a provided filename e.g. data.yaml.

```sh
bluprint index <dirname> <config.yaml>
```

In [7]:
import os
from pathlib import Path
from omegaconf import OmegaConf

In [10]:
cfg = {}

for root, dirs, files in os.walk('../demo/'):
	print(f'Root: {root}')
	print(f'Dirs: {dirs}')
	for file in files:
		file_path = Path(root) / file
		print(f'- File: {file_path}')

Root: ../demo/
Dirs: ['data', 'project', 'conf', 'notebooks']
- File: ../demo/.gitignore
Root: ../demo/data
Dirs: []
- File: ../demo/data/example_data.csv
Root: ../demo/project
Dirs: []
- File: ../demo/project/example.py
Root: ../demo/conf
Dirs: []
- File: ../demo/conf/workflow.yaml
- File: ../demo/conf/config.yaml
Root: ../demo/notebooks
Dirs: []
- File: ../demo/notebooks/example_notebook.ipynb
